In [11]:
import pandas as pd
import pmdarima as pm
from datetime import datetime

# Lee el archivo como un DataFrame
#df = pd.read_csv('/Users/micaelabassan/Desktop/fundamentos/Labo 3/sell-in.txt', delimiter='\t',  decimal='.')  # Cambia el delimitador si es necesario
df  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
# Muestra las primeras filas del DataFrame
print(df.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [12]:
product_ids_validos = df_pid_validos['product_id'].tolist()

In [13]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [15]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = df[df['product_id'].isin(product_ids_validos)]

In [16]:
data.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


In [18]:
# Transformar el campo periodo en formato fecha para aplicar al autoarima, y orfenarlos por fecha

In [19]:
import pandas as pd
import pmdarima as pm
from pmdarima import model_selection
import numpy as np

# Convertir el campo 'periodo' a datetime
data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')

# Filtrar datos para solo tener los necesarios
data = data[['periodo', 'product_id', 'tn']]

# Inicializar un diccionario para almacenar los modelos y predicciones
predictions = {}

# Obtener la lista de productos únicos
productos = data['product_id'].unique()

# Loop through each product
for producto in productos:
    # Filtrar los datos para el producto actual
    product_data = data[data['product_id'] == producto]
    
    # Asegurarse de que los datos están ordenados por el periodo
    product_data = product_data.sort_values('periodo')
    
    # Dividir los datos en entrenamiento y prueba
    train, test = model_selection.train_test_split(product_data['tn'], train_size=0.8)
    
    # Ajustar el modelo auto_arima
    model = pm.auto_arima(train, seasonal=False, stepwise=True, trace=True)
    
    # Predecir en los datos de prueba
    preds, conf_int = model.predict(n_periods=len(test), return_conf_int=True)
    
    # Guardar las predicciones y el intervalo de confianza
    predictions[producto] = {
        'predictions': preds,
        'conf_int': conf_int,
        'test': test.values
    }

# Mostrar los resultados para el primer producto como ejemplo
first_product = list(predictions.keys())[0]
print(f"Predicciones para el producto {first_product}:")
print(predictions[first_product])

C:\Users\hbasilio\AppData\Local\Temp\ipykernel_10852\3398301799.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=3.50 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=604.067, Time=0.08 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=567.048, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=574.586, Time=0.24 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=538.959, Time=0.16 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=513.752, Time=0.12 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=500.679, Time=0.21 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=465.952, Time=0.58 sec
 ARIMA(5,0,1)(0,0,0)[0]             : AIC=inf, Time=3.90 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=inf, Time=3.23 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : AIC=281.110, Time=0.92 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=281.146, Time=0.41 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : AIC=282.436, Time=3.59 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : AIC=282.494, Time=1.96 sec

Best model:  ARIMA(5,0,0)(0,0,0)[0] intercept
Total fit ti

C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


KeyboardInterrupt: 

In [20]:
# Convertir la lista de predicciones a un DataFrame
predictions_df = pd.DataFrame(predictions)

# Guardar las predicciones en un archivo CSV
predictions_df.to_csv('predicciones_febrero_2019.csv', index=False)

In [21]:
# Convertir el campo 'periodo' a datetime usando .loc para evitar el SettingWithCopyWarning
data.loc[:, 'periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')

# Filtrar datos para solo tener los necesarios
data = data[['periodo', 'product_id', 'tn']]

# Inicializar una lista para almacenar las predicciones
predictions = []

# Obtener la lista de productos únicos
productos = data['product_id'].unique()

# Definir el mes objetivo para la predicción
target_period = pd.to_datetime('202002', format='%Y%m')

# Loop through each product
for producto in productos:
    # Filtrar los datos para el producto actual
    product_data = data[data['product_id'] == producto].copy()
    
    # Asegurarse de que los datos están ordenados por el periodo
    product_data = product_data.sort_values('periodo')
    
    # Establecer el índice de tiempo
    product_data.set_index('periodo', inplace=True)
    
    # Ajustar el modelo auto_arima a toda la serie temporal
    model = pm.auto_arima(product_data['tn'], seasonal=False, stepwise=True, trace=False)
    
    # Calcular el número de períodos para predecir hasta febrero de 2019
    last_period = product_data.index.max()
    n_periods = (target_period.year - last_period.year) * 12 + (target_period.month - last_period.month)
    
    # Predecir los tn para el período objetivo
    if n_periods > 0:
        preds, conf_int = model.predict(n_periods=n_periods, return_conf_int=True)
        pred_tn = preds[-1]  # Obtener la predicción para febrero de 2019
    else:
        pred_tn = model.predict(n_periods=1)[0]  # Si el último periodo es enero 2019, predecir el siguiente
    
    # Agregar la predicción a la lista
    predictions.append({'product_id': producto, 'tn': pred_tn})

# Convertir la lista de predicciones a un DataFrame
predictions_df = pd.DataFrame(predictions)

# Guardar las predicciones en un archivo CSV
predictions_df.to_csv('predicciones_febrero_2020.csv', index=False)

C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


KeyError: -1